#### Simple Gen AI APP Using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader=WebBaseLoader("https://langchain-ai.github.io/langgraph/concepts/#langgraph")
loader


In [13]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/concepts/#langgraph', 'title': 'Concepts', 'description': 'Conceptual Guide for LangGraph', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\nConcepts\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            \n              Concepts\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n        \n  \n    \n  \n  Home\n\n      \n\n\n\n          \n  \n    \n  \n  Tutorials\n\n        \n\n\n\n          \n  \n    \n  \n  How-to Guides\n\n        \n\n\n\n          \n  \n    \n  \n  Conceptual Guides\n\n        \n\n\n\n          \n  \n    \n  \n  Reference\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n    \n  \n    Home\n  \n\n    \n  \n\n\n\n\n\n    \n  \n   

In [14]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [15]:
documents

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/concepts/#langgraph', 'title': 'Concepts', 'description': 'Conceptual Guide for LangGraph', 'language': 'en'}, page_content='Concepts\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            \n              Concepts\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n        \n  \n    \n  \n  Home\n\n      \n\n\n\n          \n  \n    \n  \n  Tutorials\n\n        \n\n\n\n          \n  \n    \n  \n  How-to Guides\n\n        \n\n\n\n          \n  \n    \n  \n  Conceptual Guides\n\n        \n\n\n\n          \n  \n    \n  \n  Reference\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n    \n  \n    Home\n  \n\n    \n  \n\n\n\n\n\n    \n  \n    Tutorials\n  \n\n    \n

In [16]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [18]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [19]:
vectorstoredb


In [20]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content


"Deployment Options¶\n\nSelf-Hosted Lite: A free (up to 1 million nodes executed), limited version of LangGraph Platform that you can run locally or in a self-hosted manner\nCloud SaaS: Hosted as part of LangSmith.\nBring Your Own Cloud: We manage the infrastructure, so you don't have to, but the infrastructure all runs within your cloud.\nSelf-Hosted Enterprise: Completely managed by you.\n\nComments\n\n\n\n\n\n\n\n\n\n\n  Back to top\n\n\n\n\n\n\n\n\n\n\n                Previous\n              \n\n                MULTIPLE_SUBGRAPHS\n              \n\n\n\n\n\n                Next\n              \n\n                Why LangGraph?\n              \n\n\n\n\n\n\n\n\n\n  \n  \n    Made with\n    \n      Material for MkDocs Insiders"

In [21]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [22]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020EF02AA8F0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020ED840A470>, root_client=<openai.OpenAI object at 0x0000020EF02AAAA0>, root_async_client=<openai.AsyncOpenAI object at 0x0000020EF02AA320>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_do

In [23]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'LangSmith has two usage limits: total traces and extended traces. These limits correspond to the two metrics tracked on the usage graph.'

In [24]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [25]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [26]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020EF02A9630>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [27]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'What are the deployment options mentioned for LangGraph Platform based on the provided context?\n\nThe deployment options mentioned for LangGraph Platform are:\n\n1. Self-Hosted Lite: A free, limited version that you can run locally or in a self-hosted manner.\n2. Cloud SaaS: Hosted as part of LangSmith.\n3. Bring Your Own Cloud: Infrastructure is managed by LangGraph but runs within your cloud.\n4. Self-Hosted Enterprise: Completely managed by you.'

In [28]:

response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/concepts/#langgraph', 'title': 'Concepts', 'description': 'Conceptual Guide for LangGraph', 'language': 'en'}, page_content="Deployment Options¶\n\nSelf-Hosted Lite: A free (up to 1 million nodes executed), limited version of LangGraph Platform that you can run locally or in a self-hosted manner\nCloud SaaS: Hosted as part of LangSmith.\nBring Your Own Cloud: We manage the infrastructure, so you don't have to, but the infrastructure all runs within your cloud.\nSelf-Hosted Enterprise: Completely managed by you.\n\nComments\n\n\n\n\n\n\n\n\n\n\n  Back to top\n\n\n\n\n\n\n\n\n\n\n                Previous\n              \n\n                MULTIPLE_SUBGRAPHS\n              \n\n\n\n\n\n                Next\n              \n\n                Why LangGraph?\n              \n\n\n\n\n\n\n\n\n\n  \n  \n    Made with\n    \n      Material for 

In [29]:
response['context']

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/concepts/#langgraph', 'title': 'Concepts', 'description': 'Conceptual Guide for LangGraph', 'language': 'en'}, page_content="Deployment Options¶\n\nSelf-Hosted Lite: A free (up to 1 million nodes executed), limited version of LangGraph Platform that you can run locally or in a self-hosted manner\nCloud SaaS: Hosted as part of LangSmith.\nBring Your Own Cloud: We manage the infrastructure, so you don't have to, but the infrastructure all runs within your cloud.\nSelf-Hosted Enterprise: Completely managed by you.\n\nComments\n\n\n\n\n\n\n\n\n\n\n  Back to top\n\n\n\n\n\n\n\n\n\n\n                Previous\n              \n\n                MULTIPLE_SUBGRAPHS\n              \n\n\n\n\n\n                Next\n              \n\n                Why LangGraph?\n              \n\n\n\n\n\n\n\n\n\n  \n  \n    Made with\n    \n      Material for MkDocs Insiders"),
 Document(metadata={'source': 'https://langchain-ai.github.io/la